In [ ]:
# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')  


<p><b><font size="200"> A Small Showcase of CREST </font></b></p>
Thank you to [Jupyter](http://jupyter.org/) and [Binder](https://mybinder.org/) for building the base of interactive CREST.

This is a preliminary implementation. It is meant as a showcase and is not yet production ready.  
The source code can be found here: https://github.com/stklik/CREST/tree/binder

## How to use this Jupyter notebook:  
select a cell (with code) and click the 'Run' button above to execute it.
If there is output, it will be shown directly underneath the cell. 
Remember that the cells can create dependencies

**Alternatively**:  
Menu 'Cell' -> 'Run all'


# First import the necessary libraries

In [ ]:
# import all model concepts
from src.model import *

# import the simulator
from src.simulator.simulator import Simulator

# import the plotting libraries that can visualise the CREST systems
from src.ui import elk, dotter

## Define the resources that we use in the system

In [ ]:
# define the resources in a class, so they are bundled. This is syntactic sugar.
# it is important however that none of the resources has the same name as any of the ports!

# currently supported value domains: REAL, INTEGER and lists of discrete values
class Resources(object):
    electricity = Resource("Watt", REAL)
    switch = Resource("switch", ["on", "off"])
    light = Resource("Lumen", INTEGER)
    counter = Resource("Count", INTEGER)
    time = Resource("minutes", REAL)
    celsius = Resource("Celsius", REAL)
    fahrenheit = Resource("Fahrenheit", REAL)
    

## Let's define an entity
Entities derive from the basic 'Entity' class.  
They can define:
 - inputs, outputs and local variables
 - states and the current state
 - transitions between states
 - influences between ports (incl. transformations)
 - updates related to a port, which continuously evaluate while in a certain state
 - actions, which are updates that are executed when a transition is fired (semantically with a dt of 0, here we don't even use the parameter)

In [ ]:

# this is a definition of a new Entity type
# it derives from the Entity base class
class LightElement(Entity):
    # we define ports - each has a resource and an initial value
    electricity = Input(resource=Resources.electricity, value=0)
    light = Output(resource=Resources.light, value=0)
    
    # automata states - don't forget to specify one as the current state
    on = State()
    off = current = State()
    
    # transitions and guards (as lambdas)
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.electricity.value >= 100))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.electricity.value < 100))
    
    # update functions are related to a state and define the value of a port
    @update(state=on, target=light)
    def set_light_on(self, dt=0):
        return 800

    @update(state=off, target=light)
    def set_light_off(self, dt=0):
        return 0

## Plotting
We use the *dotter* module to create an interactive system representation 
(based on the [`mxGraph`](https://jgraph.github.io/mxgraph/) diagram library and the [`elkjs`](https://github.com/OpenKieler/elkjs) layout engine.)
The layout and production of the HTML version takes a few seconds 

This interactive diagramming is continuously extended. The currently supported features are:

- Move objects around if the automatic layout does not provide an sufficient result.
- Select ports and states to see their outgoing arcs (blue) and incoming arcs (red).
- Hover over transitions, influences and actions to display their name and short summary.
- Double click on transitions, influences and actions you will see their source code.
- There is a *hot corner* on the top left of each entity. You can double-click it to collapse the entity. This feature is useful for CREST diagrams with many entities. *Unfortunately a software issue prevents the expand/collapse icon not to be displayed. It still works though (notice your cursor changing to a pointer)*

 
**GO AHEAD AND TRY IT**

In [ ]:
elk.plot(LightElement())

### Alternative plotting
There exists a fast, static plotter based on `graphviz` and `dot`.
You can use the `dotter` module to plot a quick picture. BUT you cannot modify it or influence the output.

In [ ]:
dotter.plot(LightElement())

## Define another entity (without transitions)
Note that this one does only have one state and no transitions.

In [ ]:
class HeatElement(Entity):
    heat = Output(resource=Resources.celsius, value=0)
    electricity = Input(resource=Resources.electricity, value=0)
    
    state = current = State() # the only state of this entity
    
    # define an influence function
    @influence(source=electricity, target=heat)
    def heat_output(value):
        return value / 100

# show us what it looks like
elk.plot(HeatElement())

# Logical Entities

These entities are just like any other entity.  
We however specify explicitly that they do not have real world counterparts.

In [ ]:
# a logical entity (this one sums two values)
class Adder(LogicalEntity):
    heat_in = Input(resource=Resources.celsius, value=0)
    room_temp_in = Input(resource=Resources.celsius, value=22)
    temperature = Output(resource=Resources.celsius, value=22)
    
    state = current = State()
    @update(state=state, target=temperature)
    def add(self, dt):
        return self.heat_in.value + self.room_temp_in.value
    
elk.plot(Adder())  # try adding the display option 'show_update_ports=True' and see what happens!

# A more complex entity (with subentities)
The GrowLamp is a standard entity just like the ones above, except...  
... it defines subentities !!

In [ ]:
class GrowLamp(Entity):
    # interface
    electricity = Input(resource=Resources.electricity, value=0)
    switch = Input(resource=Resources.switch, value="off")
    room_temperature = Input(resource=Resources.fahrenheit, value=71.6)
    light = Output(resource=Resources.light, value=3.1415*1000) # note that these are bogus values
    temperature = Output(resource=Resources.celsius, value=4242424242) # yes, nonsense...
    
    # states & transitions
    on = State()
    off = current = State()
    error = State()
    
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch.value == "on" and self.electricity.value >= 100))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch.value == "off" or self.electricity.value < 100))
    on_time = Local(resource=Resources.time, value=0)
    switch_on_counter = Local(resource=Resources.counter, value=0)
    
    
    # transition to error state if the lamp ran for more than 1000.5 time units
    @transition(source=on, target=error)
    def to_error(self):
        return self.on_time.value >= 1000.5
    
    # subentitites are defined HERE !!!
    """ - - - - - - - - - - - - - - - - - """
    lightelement = LightElement()
    heatelement = HeatElement()
    adder = Adder()
    """ - - - - - - - - - - - - - - - - - """
    
    @update(state=on, target="lightelement.electricity")
    def update_light_elec(self, dt):
        return 100
    
    @update(state=on, target="heatelement.electricity")
    def update_heat_elec(self, dt):
        return self.electricity.value - 100
        
    @update(state=on, target=on_time)
    def update_time(self, dt):
        return self.on_time.value + dt
        
    @influence(source=room_temperature, target=adder.room_temp_in)
    def celsius_to_fahrenheit(value):
        return (value - 32) * 5 / 9
    
    # we can also define updates and influences with lambda functions... 
    heat_to_add = Influence(source=heatelement.heat, target=adder.heat_in, function=(lambda val: val))
    
    # if the lambda function doesn't do anything (like the one above) we can omit it...
    add_to_temp = Influence(source=adder.temperature, target=temperature)
    light_to_light = Influence(source=lightelement.light, target=light)
    
    # let's add an action that counts the number of times we switch to state "on"
    # Note that actions do not have a dt
    @action(transition=off_to_on, target=switch_on_counter)
    def count_switching_on(self):
        return self.switch_on_counter.value + 1

# create an instance!
gl = GrowLamp()
elk.plot(gl)

## We are now going to use the Simulator
A simulator is initialized with a root-entity and a time domain.
For convenience we can define which plotter to use too.

In [ ]:
sim = Simulator(gl)
sim.plot() # the plotting should be the same as above -- except for the current time, which is set to 0 (check the root-entity's label)

## Stabilisation
The simulator will execute the system until it is stable. 
That is, until there is a point where no more transitions can be triggered and all updates/influences/actions have been executed. 
In our example look specifically at the **output** values, they are now correct (instead of the dummy default values)!!

*Note: the value of room_temp_in is 21.999999...  This is a result of the calculation: (71.6-32)*5/9*
It has to do with Python's floating point calculations. 
The next version of CREST will use SymPy to perform calculations symbolically and have precise results.*

In [ ]:
sim.stabilise()
sim.plot()

## Modification of input values

When we modify input port values, we see that nothing happens except what we explicitly changed.  

In [ ]:
# modify the growlamp instance's inputs directly, the simulator points to that object and will use it
gl.electricity.value = 500
gl.switch.value = "on"
sim.plot()

It actually takes a *stabilise* step to propagate the values

In [ ]:
sim.stabilise()
sim.plot()

## Time advance
The *advance(dt)* function can forward time.  
Below we advance 500 time steps. The effect is that the time is now (t=500) and that on_time has the value of 500 too!

In [ ]:
sim.advance(500)
sim.plot()

## Next transition time
The simulation of time advance is based on the calculation of the next transition time.

The simulator can calculate this time by analysing system's updates and influences.  
Currently, only a limited set of features is supported in update/influence functions: variable assignments, arithmetics

*More features (conditional statements & expressions) are being developped but have to be tested first*

The example below states that the error transition will be triggered after 500.5 time steps (i.e. when on_time == 1000.5)

In [ ]:
gl = GrowLamp()
gl.electricity.value = 500
gl.switch.value = "on"

sim = Simulator(gl)
sim.stabilise()
sim.get_next_transition_time()

## Automatic firing of transitions
Let's assume we want to advance more than the necessary 1000.5 time steps, what happens then?
The semantics prescribe to first advance 1000.5 time steps, then fire all transitions and updates (stabilise the system), then advance the rest.

In [ ]:
gl = GrowLamp()
gl.electricity.value = 500
gl.switch.value = "on"

sim = Simulator(gl)
sim.stabilise()
sim.get_next_transition_time()


sim.advance(1500)  # try to modify the value here to 1000 and see that no transition happens, but if you add a value >= 1000.5 the transition is fired
sim.plot()

## Different time scales
By default the simulator will run in real-time.
However, it can be configured to use integers as time unit.
Below we have a simulator that produces two different outputs, depending on the time unit chosen.
Effectively, the system changes to the *error* state when the *on_time* reaches 1000.5 or more.

In **integer** mode the simulator will calculate that a transition happens after **1001** time units, while the **real**-valued simulator will calculate the precise point in time (**1000.5**)


In [ ]:
growlamp = GrowLamp()                 # create a growlamp instance and change inputs so it is in state *on* after stabilisation
growlamp.electricity.value = 500
growlamp.switch.value = "on"
simulator = Simulator(growlamp)
simulator.stabilise()                 # stabilise the system so it's in good state
simulator.timeunit = REAL
simulator.get_next_transition_time()  # should say 1000.5 time steps

**Now: We will use the same simulator but with a different time unit to show that the result is truly different.**

In [ ]:
growlamp = GrowLamp()                 # create a growlamp instance and change inputs so it is in state *on* after stabilisation
growlamp.electricity.value = 500
growlamp.switch.value = "on"
simulator = Simulator(growlamp)
simulator.stabilise()                 # stabilise the system so it's in good state
simulator.timeunit = INTEGER
simulator.get_next_transition_time()  # should say 1001 time steps